In [73]:
import xarray as xr
import gfdl_utils as gu
import glob
import cftime
import numpy as np
import esm4ppe
from dask.diagnostics import ProgressBar

# Load data

In [65]:
basedir = '/archive/kbr/fre/verona/ESM2M/20161113/ANALYSIS/'
variable = 'NPP'

## Load ensemble

In [66]:
filename = variable+'_ENS*.nc'
path = basedir+variable+'_ENSEMBLE/'+filename
# Get rid of ensmn ones
path = [p for p in glob.glob(path) if 'ensmn' not in p]

In [67]:
def preprocess(ds):
    nt = len(ds['TIME'])
    # Recover ensemble details from path
    path = ds.encoding["source"]
    filename = path.split('/')[-1]
    filenamelist = filename.split('_')
    ensembleid = filenamelist[1]
    memberid = filenamelist[2]
    if memberid[0:3]=='neg':
        sign = -1
    else:
        sign=1
    member = int(memberid[-2:])*sign
    startyear = int(filenamelist[3])
    
    # Expand init dimension
    init = cftime.DatetimeNoLeap(startyear,1,1)
    ds = ds.expand_dims({'init':[init]})
    # Expand member dimension
    ds = ds.expand_dims({'member':[member]})
    # Assign "lead" coordinate
    ds = ds.rename({'TIME':'lead'}).assign_coords({'lead':np.arange(1,1+nt)})
    ds['lead'].attrs['units']='months'
        
    ds = ds.chunk({'lead':-1})
    return ds

In [68]:
ds = xr.open_mfdataset(path,preprocess=preprocess)[variable]

## Load control

In [69]:
controlnames = {'NPP':'NPP_100'}

localdir = 'CONTROL/'+variable+'/'
filename = controlnames[variable]+'_0001_0300.nc'
path = basedir+localdir+filename

In [70]:
control = xr.open_dataset(path).rename({'TIME':'time'})[variable]

# Calculate PPP

In [71]:
ppp = esm4ppe.calc_ppp(ds,control,groupby=None,frequency='monthly',nlead=120)

# Load and save PPP

In [77]:
savedir = '/work/gam/projects/esm2m_ppe/data/processed/verify/'
localdir = 'verify.metric-ppp/'
filename = variable+'.monthly.nc'
savepath = savedir+localdir+filename

In [79]:
with ProgressBar():
    ppp = ppp.load()
ppp.to_netcdf(savepath)